In [20]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='.',name='PROTEINS').shuffle()

print(f'Dataset : {dataset}')
print(f'Number of graphs : {len(dataset)}')
print(f'Number of nodes : {dataset[0].x.shape[0]}')
print(f'Number of features : {dataset.num_features}')
print(f'Number of classes : {dataset.num_classes}')

Dataset : PROTEINS(1113)
Number of graphs : 1113
Number of nodes : 31
Number of features : 3
Number of classes : 2


In [ ]:
for i in range(0,5): #확인
    print(dataset[i])
# y=[1] : 노드 분류가 아니라 그래프 분류. (노드 분류라면 x의 갯수와 라벨의 갯수가 동일해야 함)

Data(edge_index=[2, 144], x=[42, 3], y=[1])
Data(edge_index=[2, 52], x=[14, 3], y=[1])
Data(edge_index=[2, 56], x=[20, 3], y=[1])
Data(edge_index=[2, 30], x=[8, 3], y=[1])
Data(edge_index=[2, 74], x=[17, 3], y=[1])


In [14]:
train_dataset = dataset[:int(len(dataset)*0.8)]
val_dataset = dataset[int(len(dataset)*0.8):int(len(dataset)*0.9)]
test_dataset = dataset[int(len(dataset)*0.9):]

print(f'Training Set : {len(train_dataset)} graphs')
print(f'Val Set : {len(val_dataset)} graphs')
print(f'Test Set : {len(test_dataset)} graphs')

Training Set : 890 graphs
Val Set : 111 graphs
Test Set : 112 graphs


In [15]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=True)

print('Train Loader')
for i,batch in enumerate(train_loader):
    print(f'Batch {i} : {batch}')

print('Val Loader')
for i,batch in enumerate(val_loader):
    print(f'Batch {i} : {batch}')

print('Test Loader')
for i,batch in enumerate(test_loader):
    print(f'Batch {i} : {batch}')


Train Loader
Batch 0 : DataBatch(edge_index=[2, 7294], x=[1973, 3], y=[64], batch=[1973], ptr=[65])
Batch 1 : DataBatch(edge_index=[2, 9678], x=[2664, 3], y=[64], batch=[2664], ptr=[65])
Batch 2 : DataBatch(edge_index=[2, 9174], x=[2397, 3], y=[64], batch=[2397], ptr=[65])
Batch 3 : DataBatch(edge_index=[2, 8778], x=[2372, 3], y=[64], batch=[2372], ptr=[65])
Batch 4 : DataBatch(edge_index=[2, 9684], x=[2650, 3], y=[64], batch=[2650], ptr=[65])
Batch 5 : DataBatch(edge_index=[2, 10856], x=[2960, 3], y=[64], batch=[2960], ptr=[65])
Batch 6 : DataBatch(edge_index=[2, 8166], x=[2132, 3], y=[64], batch=[2132], ptr=[65])
Batch 7 : DataBatch(edge_index=[2, 9232], x=[2480, 3], y=[64], batch=[2480], ptr=[65])
Batch 8 : DataBatch(edge_index=[2, 7600], x=[1990, 3], y=[64], batch=[1990], ptr=[65])
Batch 9 : DataBatch(edge_index=[2, 10478], x=[2745, 3], y=[64], batch=[2745], ptr=[65])
Batch 10 : DataBatch(edge_index=[2, 10654], x=[2863, 3], y=[64], batch=[2863], ptr=[65])
Batch 11 : DataBatch(edge_

In [33]:
import torch
torch.manual_seed(0) #토치의 난수생성 시드 고정
import torch.nn.functional as F
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GINConv
from torch_geometric.nn import global_add_pool

class GIN(torch.nn.Module):
    def __init__(self,dim_h):
        super(GIN,self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(dataset.num_node_features,dim_h),
                              BatchNorm1d(dim_h),
                              ReLU(),
                              Linear(dim_h,dim_h),
                              ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(dim_h,dim_h),
                              BatchNorm1d(dim_h),
                              ReLU(),
                              Linear(dim_h,dim_h),
                              ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(dim_h,dim_h),
                              BatchNorm1d(dim_h),
                              ReLU(),
                              Linear(dim_h,dim_h),
                              ReLU()))
        self.lin1 = Linear(dim_h*3,dim_h*3)
        self.lin2 = Linear(dim_h*3,dataset.num_classes)

    def forward(self,x,edge_index,batch):
        # 노드 임베딩
        h1 = self.conv1(x,edge_index) # 1홉의 정보를 반영한 모든 노드 임베딩
        h2 = self.conv2(h1,edge_index) # 2홉
        h3 = self.conv3(h2,edge_index) # 3홉

        h1 = global_add_pool(h1,batch) # 배치는 그래프의 구분 정보를 암시
        h2 = global_add_pool(h2,batch)
        h3 = global_add_pool(h3,batch)

        #그래프 임베딩으로 표현
        h = torch.cat((h1,h2,h3),dim=1)

        #분류 신경망
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h,p=0.5,training=self.training)
        h = self.lin2(h)
        
        return h


In [34]:
def accuracy(y_pred,y):
    return ((y_pred==y).sum() / len(y)).item()

In [39]:
@torch.no_grad()
def test(model,loader):
    criterion = torch.nn.CrossEntropyLoss()
    model.eval()
    loss,acc = 0,0
    for data in loader:
        out = model(data.x,data.edge_index,data.batch)
        loss += criterion(out,data.y) / len(loader)
        acc += accuracy(out.argmax(dim=1),data.y) / len(loader)
    
    return loss,acc

In [43]:
def train(model,loader):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
    epochs = 100

    model.train()
    for epoch in range(epochs+1):
        total_loss,total_acc,val_loss,val_acc = 0,0,0,0

        for data in loader:
            optimizer.zero_grad()
            out = model(data.x,data.edge_index,data.batch)
            loss = criterion(out,data.y) #하나의 로스, 스칼라 값이 나옴. (평균)
            total_loss += loss.item() / len(loader)
            total_acc += accuracy(out.argmax(dim=1),data.y) / len(loader)
            loss.backward()
            optimizer.step()

        val_loss,val_acc = test(model,val_loader)
        
        if epoch%20==0:
            print(f'Epoch : {epoch:>3} | Train_loss : {total_loss:.3f} | Train_acc : {total_acc*100:.3f} | Val_loss : {val_loss:.3f} | Val_acc : {val_acc*100:.3f}')

    return model            



In [44]:
gin = GIN(dim_h=32)
print(gin)

GIN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): ReLU()
  ))
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): ReLU()
  ))
  (lin1): Linear(in_features=96, out_features=96, bias=True)
  (lin2): Linear(in_features=96, out_features=2, bias=True)
)


In [45]:
gin_model = train(gin,train_loader)

Epoch :   0 | Train_loss : 1.643 | Train_acc : 58.297 | Val_loss : 0.602 | Val_acc : 64.229
Epoch :  20 | Train_loss : 0.505 | Train_acc : 77.394 | Val_loss : 0.588 | Val_acc : 75.798
Epoch :  40 | Train_loss : 0.486 | Train_acc : 77.159 | Val_loss : 0.568 | Val_acc : 77.078
Epoch :  60 | Train_loss : 0.482 | Train_acc : 76.736 | Val_loss : 0.510 | Val_acc : 73.454
Epoch :  80 | Train_loss : 0.463 | Train_acc : 78.702 | Val_loss : 0.528 | Val_acc : 74.734
Epoch : 100 | Train_loss : 0.474 | Train_acc : 76.970 | Val_loss : 0.554 | Val_acc : 74.734
